In [1]:
import pandas as pd
import cobra
from cobra import Model, Reaction, Metabolite

In [2]:
reactions = pd.read_excel('reactions_extended.xlsx')
reactions.head(2)

,Abbreviation,Name,Reaction,Reactions with MetNames,GPR,Genes,Protein,Subsystems,Reversible,Lower bound,Upper bound,Objective,Confidence Score,EC. Number,Notes,References
0,rxn03253,acyl-CoA dehydrogenase (decanoyl-CoA),cpd00015[c] + cpd03128[c] -> cpd00982[c] + cpd...,FAD + Decanoyl-CoA -> FADH2 + (2E)-Decenoyl-CoA,((PA14_52900) or (PA14_06600) or (PA14_31580)),PA14_52900 PA14_06600 PA14_31580,NaN,Fatty acid metabolism,0,0.0,1000.0,0,NaN,NaN,*JAB 06/23/14 - switched out NAD for FAD based...,"Guzik, M. W., Narancic, T., Ilic-Tomic, T., Vo..."
1,rxn02720,acyl-CoA dehydrogenase (dodecanoyl-CoA),cpd00015[c] + cpd01260[c] -> cpd00982[c] + cpd...,FAD + Lauroyl-CoA -> FADH2 + (2E)-Dodecenoyl-CoA,((PA14_52900) or (PA14_06600) or (PA14_31580)),PA14_52900 PA14_06600 PA14_31580,NaN,Fatty acid metabolism,0,0.0,1000.0,0,NaN,NaN,*JAB 06/23/14 - switched out NAD for FAD based...,"Guzik, M. W., Narancic, T., Ilic-Tomic, T., Vo..."


In [3]:
reactions = reactions.drop(['Reactions with MetNames', 'Genes', 'Protein', 'Confidence Score'], axis=1)
reactions.head(2)

,Abbreviation,Name,Reaction,GPR,Subsystems,Reversible,Lower bound,Upper bound,Objective,EC. Number,Notes,References
0,rxn03253,acyl-CoA dehydrogenase (decanoyl-CoA),cpd00015[c] + cpd03128[c] -> cpd00982[c] + cpd...,((PA14_52900) or (PA14_06600) or (PA14_31580)),Fatty acid metabolism,0,0.0,1000.0,0,NaN,*JAB 06/23/14 - switched out NAD for FAD based...,"Guzik, M. W., Narancic, T., Ilic-Tomic, T., Vo..."
1,rxn02720,acyl-CoA dehydrogenase (dodecanoyl-CoA),cpd00015[c] + cpd01260[c] -> cpd00982[c] + cpd...,((PA14_52900) or (PA14_06600) or (PA14_31580)),Fatty acid metabolism,0,0.0,1000.0,0,NaN,*JAB 06/23/14 - switched out NAD for FAD based...,"Guzik, M. W., Narancic, T., Ilic-Tomic, T., Vo..."


# Use libSBML for adding reaction annotations

In [4]:
from libsbml import * 
reader = SBMLReader()
document = reader.readSBMLFromFile('created_PA14.xml')
model = document.getModel()

In [5]:
print('Species:', model.getNumSpecies())
print('Reactions: ', model.getNumReactions())
print('Compartments: ', model.getNumCompartments())

Species: 1295
Reactions:  1518
Compartments:  2


In [6]:
reaction_list = model.getListOfReactions()

### Read in the BiGG dictionary

In [8]:
bigg_dict = {}
with open('final_bigg_dict.txt') as f:
    for line in f:
        (key, val) = line.split('\t')
        key = key.replace(':', '')
        #val = val.replace('[', '')
        #val = val.replace(']', '')
        #val = val.replace("'", '')
        val = val.replace('\n', '')
        #val = [val]
        #print(key, val)
        bigg_dict[key] = val
print(len(bigg_dict))
#print(bigg_dict)

792


### Read in the KEGG dictionary

In [9]:
kegg_dict = {}
with open('final_kegg_dict.txt') as f:
    for line in f:
        (key, val) = line.split('\t')
        key = key.replace(':', '')
        val = val.replace('[', '')
        val = val.replace(']', '')
        val = val.replace("'", '')
        val = val.replace('\n', '')
        val = [val]
        #print(key, val)
        kegg_dict[key] = val
print(len(kegg_dict))
#print(kegg_dict)

958


In [9]:
'''Testing'''
react_test = reaction_list[0].getId()[2:]
print(react_test)
val_test = kegg_dict['rxn00007'][0]
val_test = val_test.split(',')
for val in val_test:
    val = val.strip()
    val = val[6:-1]
    print(val)
#val_test = val_test[6:-1]
#val_test = val_test.replace('"', '')
#val_test = val_test.replace('KEGG:', '')
print(val_test)

rxn03253
R00010
R06103
['"KEGG:R00010"', ' "KEGG:R06103"']


### Adding the KEGG Reaction Identifier as Annotations

In [10]:
success = 0
bigg_ids = 0
for react in reaction_list:
    react_id = react.getId()[2:]     #get the id of the reaction, without the Prefix 'R_'
    if react_id in kegg_dict.keys():
        val = kegg_dict[react_id][0]
        #val = val[6:-1]
        if not val: 
            continue
        else: 
            if ',' in val:
                #more_than_one += 1
                val = val.split(',')
                for val_part in val:
                    val_part = val_part.strip()
                    val_part = val_part[6:-1]
                    
                    bigg_ids += 1
                    
                    cv = CVTerm()
                    cv.setQualifierType(BIOLOGICAL_QUALIFIER)
                    cv.setBiologicalQualifierType(BQB_IS)
                    cv.addResource('http://identifiers.org/kegg.reaction/'+val_part)
                    
                    status = react.addCVTerm(cv)
                    if status == LIBSBML_OPERATION_SUCCESS:
                        success += 1
                    else:
                        print(react, val)

            else: 
                bigg_ids += 1
                val = val[6:-1]
                cv = CVTerm()
                cv.setQualifierType(BIOLOGICAL_QUALIFIER)
                cv.setBiologicalQualifierType(BQB_IS)
                cv.addResource('http://identifiers.org/kegg.reaction/'+val)

                status = react.addCVTerm(cv)
                if status == LIBSBML_OPERATION_SUCCESS:
                    success += 1
                else:
                    print(react, val)

print(bigg_ids)           
print(success)

664
664


In [11]:
newdocument = model.getSBMLDocument()
writeSBMLToFile(newdocument,'created_PA14.xml') # 1 means success, 0 means failure

1

### Adding the BiGG Reaction Identifiers as Annotations

In [12]:
document = reader.readSBMLFromFile('created_PA14.xml')
model = document.getModel()

In [13]:
reaction_list = model.getListOfReactions()

In [14]:
success = 0
bigg_ids = 0
for react in reaction_list:
    react_id = react.getId()[2:]     #get the id of the reaction, without the Prefix 'R_'
    if react_id in bigg_dict.keys():
        #val = bigg_dict[react_id][0] #With the original dictionary, not the manually edited
        val = bigg_dict[react_id]
        #val = val[6:-1]
        if not val: 
            continue
        else: 
            if ',' in val:
                val = val.split(',')
                for val_part in val:
                    val_part = val_part.strip()
                    val_part = val_part[6:-1]
                    
                    bigg_ids += 1
                    
                    cv = CVTerm()
                    cv.setQualifierType(BIOLOGICAL_QUALIFIER)
                    cv.setBiologicalQualifierType(BQB_IS)
                    cv.addResource('http://identifiers.org/bigg.reaction/'+val_part)
                    
                    status = react.addCVTerm(cv)
                    if status == LIBSBML_OPERATION_SUCCESS:
                        success += 1
                    else:
                        print(react, val)
            else: 
                bigg_ids += 1
                #val = val[6:-1] Necessary for the original bigg_dictionary, not the manually edited one
                cv = CVTerm()
                cv.setQualifierType(BIOLOGICAL_QUALIFIER)
                cv.setBiologicalQualifierType(BQB_IS)
                cv.addResource('http://identifiers.org/bigg.reaction/'+val)

                status = react.addCVTerm(cv)
                if status == LIBSBML_OPERATION_SUCCESS:
                    success += 1
                else:
                    print(react, val)

print(bigg_ids)           
print(success)

792
792


In [15]:
newdocument = model.getSBMLDocument()
writeSBMLToFile(newdocument,'created_PA14.xml') # 1 means success, 0 means failure

1

## Looking for missing identifiers

In [16]:
PA14_model = cobra.io.read_sbml_model('created_PA14.xml')
#PA14_model.optimize()

In [17]:
found = 0
not_found = 0
more_than_one = 0
more_than_one_dict = {}
not_found_list = []

for react in PA14_model.reactions:
    if react.id in bigg_dict.keys():
        found += 1
        if ',' in bigg_dict[react.id][0]:
            more_than_one += 1
            more_than_one_dict[react.id] = bigg_dict[react.id]
    else: 
        not_found += 1
        not_found_list.append(react.id)
        
        
        
print(found)
print(not_found)
print(more_than_one)

792
726
0


In [18]:
print(not_found_list[:10])

['rxn01387', 'rxn13812', 'rJB00270', 'rxn00779', 'rxn13726', 'rJB00264', 'rJB00276', 'rxn05354', 'rxn05351', 'rxn05436']
